# Mash pH model (Henderson–Hasselbalch)

**Doel.** Een eerste, minimalistisch rekenmodel om te begrijpen hoe pH in de maisch verschuift door (i) **zuur-toevoeging** (melkzuur) en (ii) **bufferende systemen** (vereenvoudigd).

> Dit notebook is didactisch: het is **geen volledige waterchemie- of moutbufferings­simulator**. Het model is bewust eenvoudig gehouden zodat de invloed van aannames transparant blijft.


## 1. Theorie (kort)

### 1.1 Henderson–Hasselbalch
Voor een zwak zuur HA met geconjugeerde base A⁻ geldt:

\[
\mathrm{pH} = \mathrm{p}K_a + \log_{10}\left(\frac{[A^-]}{[HA]}\right)
\]

Dit geldt **voor één bufferpaar** in oplossing. In een echte maisch zijn er meerdere buffers (fosfaten, organische zuren, moutcomponenten) en is de matrix complex. In dit notebook modelleren we één **effectieve** buffer rond een gekozen \(\mathrm{p}K_a\) en een **totale buffercapaciteit**.

### 1.2 Buffercapaciteit (vereenvoudigd)
We beschrijven de buffer als een totaal concentraat:
\[
C_T = [HA] + [A^-]
\]
en een start-pH \(\mathrm{pH}_0\). Daarmee kunnen we de beginverhouding \([A^-]/[HA]\) bepalen.

Wanneer we een hoeveelheid sterk zuur (in mol H⁺) toevoegen, verschuift een deel van \(A^-\) naar \(HA\):

\[
A^- + H^+ \rightarrow HA
\]

We negeren hier (i) volume-effecten door toevoegingen, (ii) activiteit/ionsterkte, (iii) CO₂-evenwichten en (iv) mout-specifieke titratiecurven. Het doel is conceptueel inzicht.


## 2. Parameters

Pas hieronder de parameters aan. De standaardwaarden zijn plausibele *orde van grootte* waarden voor een onderwijsmodel, niet een garantie voor uw specifieke recept.

In [ ]:

# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt

# --- Algemeen ---
V_l = 20.0            # [L] effectief maischvolume (water + vloeibare fase)
pH0 = 5.40            # [-] start-pH van de maisch (zonder correctie)
pKa_eff = 6.80        # [-] effectieve pKa van de 'dominante' buffer rond dit bereik (vereenvoudiging)
CT_mM = 25.0          # [mM] totale effectieve bufferconcentratie (HA + A-)

# --- Zuurcorrectie met melkzuur (lactic acid) ---
# We modelleren melkzuur als een zwak zuur (pKa ~ 3.86), maar in het relevante pH-bereik
# gedraagt het zich grotendeels als 'zuurbron'. We gebruiken een effectieve H+ equivalentie.
# (didactische simplificatie)
lactic_solution_w_w = 0.80   # 80% (m/m) melkzuuroplossing (typisch brouwwinkelproduct)
rho_solution = 1.20          # [g/mL] dichtheid (typisch; varieert per product)
MW_lactic = 90.08            # [g/mol] molmassa melkzuur
equiv_H_per_mol = 1.0        # 1 mol melkzuur kan ~1 mol H+ leveren (conceptueel)

# Instelbare reeks melkzuur-toevoeging:
lactic_mL = np.linspace(0, 40, 81)   # [mL] 0–40 mL in stapjes


## 3. Model

1. Bepaal uit \(\mathrm{pH}_0\) en \(\mathrm{p}K_a\) de beginverhouding \([A^-]/[HA]\).  
2. Splits \(C_T\) in \([A^-]_0\) en \([HA]_0\).  
3. Zet melkzuurvolume om naar mol H⁺ equivalenten.  
4. Laat toegevoegd H⁺ reageren met \(A^-\) (tot het op is).  
5. Bereken de nieuwe pH via Henderson–Hasselbalch.

**Let op:** Als \(A^-\) volledig is opgebruikt, is het bufferpaar “uitgeput” en klopt het model niet meer (pH zal dan sterker dalen dan het model betrouwbaar kan voorspellen). We markeren daarom het punt waar \(A^-\) naar nul gaat.


In [ ]:

# --- Helperfuncties ---
def split_buffer_species(pH, pKa, CT_mM):
    """Gegeven pH en pKa: bereken [A-] en [HA] (mM) uit CT."""
    r = 10**(pH - pKa)           # r = [A-]/[HA]
    HA = CT_mM / (1 + r)
    A = CT_mM - HA
    return A, HA

def lactic_moles_H(lactic_mL, w_w=0.80, rho=1.20, MW=90.08, equiv=1.0):
    """Zet mL 80% (m/m) melkzuuroplossing om naar mol H+ equivalent (didactisch)."""
    mass_solution_g = lactic_mL * rho                    # g
    mass_lactic_g = mass_solution_g * w_w                # g melkzuur
    mol_lactic = mass_lactic_g / MW                      # mol melkzuur
    mol_H = mol_lactic * equiv
    return mol_H

# --- Startbuffer ---
A0_mM, HA0_mM = split_buffer_species(pH0, pKa_eff, CT_mM)

# --- Zuurtoevoeging omrekenen naar mM H+ in de maisch ---
mol_H_added = lactic_moles_H(lactic_mL, lactic_solution_w_w, rho_solution, MW_lactic, equiv_H_per_mol)
mM_H_added = (mol_H_added / V_l) * 1000.0   # mol/L -> mM

# --- Reactie: A- + H+ -> HA ---
A_mM = A0_mM - mM_H_added
HA_mM = HA0_mM + mM_H_added

# Grenzen: buffer kan niet negatief
buffer_exhausted = A_mM <= 0
A_mM_clipped = np.clip(A_mM, 1e-9, None)   # voorkomen deling door nul
HA_mM_clipped = np.clip(HA_mM, 1e-9, None)

# --- Henderson–Hasselbalch pH ---
pH = pKa_eff + np.log10(A_mM_clipped / HA_mM_clipped)

# Voor de regio waar buffer uitgeput is: pH-waarden zijn niet betrouwbaar; we maskeren ze
pH_masked = pH.copy()
pH_masked[buffer_exhausted] = np.nan

# Rapporteer enkele kernwaarden
print(f"Start: pH0={pH0:.2f}, pKa_eff={pKa_eff:.2f}, CT={CT_mM:.1f} mM")
print(f"Initieel: [A-]_0={A0_mM:.2f} mM, [HA]_0={HA0_mM:.2f} mM")

if np.any(buffer_exhausted):
    idx = np.argmax(buffer_exhausted)
    print(f"Buffer uitgeput vanaf ~{lactic_mL[idx]:.1f} mL melkzuur (model wordt daar voorbij onbetrouwbaar).")



## 4. Plot

De grafiek toont de voorspelde pH als functie van melkzuur-toevoeging. Na uitputting van de buffer wordt de curve onderbroken (maskering).

In [ ]:

plt.figure(figsize=(8, 4.8))
plt.plot(lactic_mL, pH_masked, marker='o', markersize=3, linewidth=1)
plt.xlabel("Toegevoegde melkzuuroplossing (mL)")
plt.ylabel("Voorspelde pH (buffer-model)")
plt.title("Mash pH vs. melkzuur-toevoeging (vereenvoudigd HH-model)")
plt.grid(True, alpha=0.3)

# Markeer startpunt
plt.scatter([0], [pH0], s=60)
plt.annotate("Start", xy=(0, pH0), xytext=(3, pH0+0.05))

# Markeer buffer-uitputting indien van toepassing
if np.any(buffer_exhausted):
    idx = np.argmax(buffer_exhausted)
    x_exh = lactic_mL[idx]
    y_exh = pH[idx-1] if idx > 0 else pH0
    plt.axvline(x_exh, linestyle='--', linewidth=1)
    plt.annotate("Buffer uitgeput", xy=(x_exh, y_exh), xytext=(x_exh+2, y_exh-0.2))

plt.tight_layout()
plt.show()


## 5. Interpretatievragen

1. **Gevoeligheid:** Verander `CT_mM` (bv. 10, 25, 50 mM). Wat gebeurt er met de benodigde melkzuurhoeveelheid om dezelfde pH-daling te krijgen?  
2. **Effectieve pKa:** Verander `pKa_eff` (bv. 6.2, 6.8, 7.2). Hoe verandert de kromming van de pH-curve rond `pH0`?  
3. **Start pH:** Zet `pH0` op 5.2 versus 5.6. Welke situatie vraagt meer zuur voor pH-correctie en waarom?  
4. **Modelgrenzen:** Zoek het punt waar de buffer “uitgeput” raakt. Wat betekent dit fysisch/chemisch? Welke extra evenwichten spelen dan mee in een echte maisch?  
5. **Kalibratie-idee:** Als u experimenteel pH meet bij 0, 10, 20 mL melkzuur: hoe zou u `CT_mM` en `pKa_eff` kunnen fitten zodat het model aansluit bij uw mout/watercombinatie?

### Uitbreiding (optioneel)
- Voeg een tweede bufferpaar toe (twee Henderson–Hasselbalch-termen) en bekijk hoe een “multi-buffer” de curve verandert.
